In [47]:

from model import ViTBERTClassifier # Adjust the import statement according to your project structure
import torch 
from torch.utils.data import Dataset, DataLoader
import captum
from captum.attr import IntegratedGradients
from captum.attr import visualization
import pandas as pd
import os
import pathlib
import torch
import pandas as pd
import transformers
from underthesea import word_tokenize
from torch.utils.data import Dataset, DataLoader
import numpy as np 

In [48]:
import torch
import torch.nn as nn
from transformers import AutoModel
from data_loader import ViTBERT
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
class DynamicClassifier(nn.Module):
    def __init__(self, 
                input_dim=768, 
                num_classes=4, 
                dropout_rate=0.25, 
                num_layers=3):
        super().__init__()
        layers = []
        for i in range(num_layers):
            output_dim = input_dim // 2
            layers.append(nn.Linear(input_dim, output_dim))
            layers.append(nn.ReLU())
            if i < num_layers - 1:  # No dropout after last layer before classification
                layers.append(nn.Dropout(dropout_rate))
            input_dim = output_dim
        layers.append(nn.Linear(input_dim, num_classes))
        layers.append(nn.Softmax(dim=1))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, feature):
        return self.model(feature)

class ViTBERTClassifier(nn.Module):
    def __init__(self, 
                pretrained_model_name, 
                num_classes=4, 
                dropout_rate=0.25, 
                num_layers=4):
        super().__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.classifier = DynamicClassifier(input_dim=self.bert.config.hidden_size,
                                            num_classes=num_classes,
                                            dropout_rate=dropout_rate,
                                            num_layers=num_layers)
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(last_hidden_state)
        return logits
    def compute_l1_loss(self, w):
        return torch.abs(w).sum()


In [49]:

model= ViTBERTClassifier(pretrained_model_name="demdecuong/vihealthbert-base-word", num_classes= 4, dropout_rate= 0.4, num_layers=3)

model.eval()
device = "cuda"

In [50]:
class ViTBERT(Dataset):
    def __init__(self, data_path, tokenizer=None):
        self.dataset = pd.read_csv(data_path)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer)

    def __getitem__(self, idx):
        text = self.dataset.iloc[idx, 0]  # Extract text from dataframe
        label = self.dataset.iloc[idx, 1]-1.0  # Extract label

        # Tokenization
        tokenized_text = self.tokenizer(text, add_special_tokens=True, max_length=100, padding='max_length', truncation=True, return_tensors='pt')

        # Extract fields from tokenized text
        input_ids = tokenized_text['input_ids'].squeeze().long()
        attention_mask = tokenized_text['attention_mask'].squeeze().long()

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.long)  # Changed dtype to torch.long

        # Return tensors
        return tokenized_text ,input_ids, attention_mask, label, text  # Return attention_mask

    def __len__(self):
        return len(self.dataset)

In [51]:
dataset = ViTBERT(data_path="/media/data3/home/khiemdd/ViTBERT/dataset/dataset_chi_ha_hieu/dataset_final_train_after.csv", 
                  tokenizer="demdecuong/vihealthbert-base-word")
dataloader = DataLoader(dataset,batch_size= 1)
tokenized_text,input_ids, attention_mask, labels, text = next(iter(dataloader))

In [52]:
print("Tokenized Text:", tokenized_text)

print("\nInput IDs:", input_ids)
print("Shape of Input IDs:", input_ids.shape)
print("Type of Input IDs:", type(input_ids))

print("\nAttention Mask:", attention_mask)
print("Shape of Attention Mask:", attention_mask.shape)
print("Type of Attention Mask:", type(attention_mask))

print("\nLabels:", labels)
print("Shape of Labels:", labels.shape)

print("\nText:", text)
print("Shape of Text:", len(text))


Tokenized Text: {'input_ids': tensor([[[    0,  1777,  6913,  1989, 10884,   280,  4823,  7682, 26496,  1989,
             76,   145,  3439,   127,   604,  1986,  1933,   509,   173,  9377,
           3372,  1730,  7656,   732,  1251,  1401,   482,   509,  9377, 27278,
           9377,    91,   827,   448,     2,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]]), 'token_type_ids': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [53]:
import torch.nn.functional as F

In [54]:
target_classes = ["rất nhẹ", "nhẹ", "vừa", "nặng"]
def predict(X_batch):
    preds = model(X_batch)
    return F.softmax(preds, dim=-1)

In [55]:
print(input_ids.shape)
print(labels.shape)

torch.Size([1, 100])
torch.Size([1])


In [56]:
input_ids.unsqueeze(0).shape

torch.Size([1, 1, 100])

In [57]:
def print_gpu_memory():
    print(f'Allocated: {torch.cuda.memory_allocated()}')
    print(f'Cached: {torch.cuda.memory_reserved()}')

# Clear cache and print memory usage
torch.cuda.empty_cache()
print_gpu_memory()

Allocated: 7786522624
Cached: 8248098816


In [58]:
model = torch.load("/media/data3/home/khiemdd/ViTBERT/src/best_model.pt")

In [59]:
model.eval()

ViTBERTClassifier(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [60]:
text = "chấn_thương sọ não máu tụ dưới màng_cứng vùng thái_dương đỉnh phải dập não xuất_huyết bao trong phải vết_thương phần_mềm vùng thái_dương đỉnh phải gãy kín đầu dưới 2 xương cẳng_chân trái gãy kín chỏm xương đốt bàn ngón i ii 3 4 chân trái do tai_nạn giao_thông tăng huyết_áp đái_tháo_đường"
tokenizer = transformers.AutoTokenizer.from_pretrained("demdecuong/vihealthbert-base-syllable")
tokenizer_text = tokenizer(text, add_special_tokens=True, max_length=100, padding='max_length', truncation=True, return_tensors='pt')
print(tokenizer_text)

{'input_ids': tensor([[    0,  1777,  6913,  1989,   764,  9950,   280, 52551,   275, 25149,
          2246,    41,  3427,  1989, 10884,  1464,    12,    41,  3372,  1730,
           275, 25149,  2246,    41,  3439,  2058,   127,   280,    76,  1986,
         24001,   732,  3439,  2058, 46689,  1986,  1933,   509,  3383,  1111,
          9428,   107,   163,   444,   732,    91,   827,   448,   128,  4019,
         11871,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0

In [61]:
input_ids = tokenizer_text["input_ids"]
attention_mask = tokenizer_text["attention_mask"]

In [62]:
input_ids = input_ids.to(torch.device("cuda:0"))
attention_mask = attention_mask.to(torch.device("cuda:0"))
model(input_ids, attention_mask)


tensor([[0.0023, 0.9709, 0.0186, 0.0082]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [63]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.start_logits, output.end_logits

In [64]:
import torch
import captum
from captum.attr import IntegratedGradients
from transformers import RobertaTokenizer


def preprocess(text):
    # Tokenize the input text and get input IDs and attention mask
    encoding = tokenizer(text, add_special_tokens=True, max_length=100, padding='max_length', truncation=True, return_tensors='pt')
    return encoding['input_ids'], encoding['attention_mask']



In [65]:
def forward_func(input_ids, attention_mask):
    # Get the output of the model
    input_ids = input_ids.to(torch.device("cuda:0"))
    attention_mask = attention_mask.to(torch.device("cuda:0"))
    input_ids = input_ids.long()
    attention_mask = attention_mask.long()
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    # Get the classification logits
    logits = outputs
    return logits


In [66]:
def explain(model, text, target_class):
    # Preprocess the input text
    input_ids, attention_mask = preprocess(text)

    # Initialize Integrated Gradients
    ig = IntegratedGradients(forward_func)

    # Attribute the integrated gradients to the input IDs
    attributions, delta = ig.attribute(inputs=input_ids, additional_forward_args=(attention_mask,), target=target_class, return_convergence_delta=True)
    
    # Decode the tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

    return attributions, tokens, delta


In [67]:
# Example text and target class (assuming target_class is 1)

target_class = 2

attributions, tokens, delta = explain(model, text, target_class)

# Display the results
for token, attribution in zip(tokens, attributions[0].tolist()):
    print(f'Token: {token}, Attribution: {attribution}')


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 10.91 GiB of which 11.31 MiB is free. Including non-PyTorch memory, this process has 10.89 GiB memory in use. Of the allocated memory 10.21 GiB is allocated by PyTorch, and 533.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [68]:
import torch
from captum.attr import IntegratedGradients
from transformers import RobertaTokenizer

# Load the model
model = torch.load("/media/data3/home/khiemdd/ViTBERT/src/best_model.pt")
model.eval()

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("demdecuong/vihealthbert-base-syllable")

def preprocess(text):
    # Tokenize the input text and get input IDs and attention mask
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    return encoding['input_ids'], encoding['attention_mask']

def forward_func(input_ids, attention_mask):
    # Ensure the input_ids are of type LongTensor and on the same device as the model
    input_ids = input_ids.long().to(model.device)
    attention_mask = attention_mask.long().to(model.device)
    # Get the output of the model
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    # Get the classification logits
    logits = outputs[0]
    print(f"Logits shape: {logits.shape}")  # Debugging line to print the shape
    return logits

def explain(model, text, target_class):
    # Preprocess the input text
    input_ids, attention_mask = preprocess(text)

    # Move input_ids and attention_mask to the same device as the model
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)

    # Initialize Integrated Gradients
    ig = IntegratedGradients(forward_func)

    # Attribute the integrated gradients to the input IDs
    attributions, delta = ig.attribute(inputs=input_ids, additional_forward_args=(attention_mask,), target=target_class, return_convergence_delta=True, allow_unused=True)
    
    # Decode the tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

    return attributions, tokens, delta

# Example text and target class (assuming target_class is 1)
text = "Example input text for explanation"
target_class = 2

attributions, tokens, delta = explain(model, text, target_class)

# Display the results
for token, attribution in zip(tokens, attributions[0].tolist()):
    print(f'Token: {token}, Attribution: {attribution}')


OSError: Can't load tokenizer for 'demdecuong/vihealthbert-base-syllable'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'demdecuong/vihealthbert-base-syllable' is the correct path to a directory containing all relevant files for a RobertaTokenizer tokenizer.